In [2]:
import os
import math
import pickle
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import cycle
from sklearn.utils import shuffle
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, \
                            r2_score, mean_poisson_deviance, mean_gamma_deviance, accuracy_score 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Without Features

In [22]:
df_eth = pd.read_csv('ETH.csv')
df_eth['Date'] = pd.to_datetime(df_eth['Date'])
df_eth = df_eth[['Date','Close']]
df_eth = df_eth[df_eth['Date'] > '2021-01-01']
df_eth = df_eth.dropna()
eth_stock = df_eth.copy()
eth_stock

,Date,Close
1150,2021-01-02,774.534973
1151,2021-01-03,975.507690
1152,2021-01-04,1040.233032
1153,2021-01-05,1100.006104
1154,2021-01-06,1207.112183
...,...,...
1801,2022-10-15,1274.871704
1802,2022-10-16,1306.296631
1803,2022-10-17,1331.713623
1804,2022-10-18,1310.447021


In [23]:
del df_eth['Date']
scaler = MinMaxScaler(feature_range=(0,1))
df_eth = scaler.fit_transform(np.array(df_eth).reshape(-1,1))

with open('scaler ETH.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# split dataset into train and test split
training_size = int(len(df_eth)*0.8)
test_size = len(df_eth) - training_size

train_data, test_data = df_eth[0:training_size,:], df_eth[training_size:len(df_eth),:1]

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 15
X_train_wof, y_train_wof = create_dataset(train_data, time_step)
X_test_wof, y_test_wof = create_dataset(test_data, time_step)

print("X_train: ", X_train_wof.shape)
print("y_train: ", y_train_wof.shape)
print("X_test: ", X_test_wof.shape)

In [ ]:
X_train_wof = X_train_wof.reshape(X_train_wof.shape[0],X_train_wof.shape[1] , 1)
X_test_wof = X_test_wof.reshape(X_test_wof.shape[0],X_test_wof.shape[1] , 1)

print("X_train: ", X_train_wof.shape)
print("X_test: ", X_test_wof.shape)

In [15]:
model_wof=Sequential()
model_wof.add(LSTM(10,input_shape=(None,1),activation="relu"))
model_wof.add(Dense(1))
model_wof.compile(loss="mean_squared_error",optimizer="sgd")
history = model_wof.fit(
                    X_train_wof,
                    y_train_wof,
                    validation_data=(X_test_wof,y_test_wof),
                    epochs=200,
                    batch_size=32,
                    verbose=1
                    )
model_wof.evaluate(X_test_wof, y_test_wof)

16/16 [==============================] - 1s 60ms/step - loss: 737034.4375 - val_loss: 1903708.1250
Epoch 32/200
16/16 [==============================] - 1s 57ms/step - loss: 736992.9375 - val_loss: 1904444.3750
Epoch 33/200
16/16 [==============================] - 1s 59ms/step - loss: 737159.9375 - val_loss: 1902613.3750
Epoch 34/200
16/16 [==============================] - 1s 58ms/step - loss: 737421.7500 - val_loss: 1892352.0000
Epoch 35/200
16/16 [==============================] - 1s 59ms/step - loss: 737239.6250 - val_loss: 1895096.8750
Epoch 36/200
16/16 [==============================] - 1s 57ms/step - loss: 737261.5000 - val_loss: 1894918.1250
Epoch 37/200
16/16 [==============================] - 1s 57ms/step - loss: 737133.1875 - val_loss: 1903608.3750
Epoch 38/200
16/16 [==============================] - 1s 56ms/step - loss: 736918.5000 - val_loss: 1905403.2500
Epoch 39/200
16/16 [==============================] - 1s 58ms/step - loss: 737255.1875 - val_loss: 1904153.3750
Epoch

KeyboardInterrupt: 

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# With Features

In [6]:
df_eth = pd.read_csv('ETH.csv')
df_eth['Date'] = pd.to_datetime(df_eth['Date'])
df_eth = df_eth[['Date','Open','High','Low','Close']]
df_eth = df_eth[df_eth['Date'] > '2021-01-01']
df_eth = df_eth.dropna()
eth_stock = df_eth.copy()
eth_stock

,Date,Open,High,Low,Close
1150,2021-01-02,730.402649,786.798462,718.109497,774.534973
1151,2021-01-03,774.511841,1006.565002,771.561646,975.507690
1152,2021-01-04,977.058838,1153.189209,912.305359,1040.233032
1153,2021-01-05,1041.498779,1129.371460,986.811279,1100.006104
1154,2021-01-06,1101.005005,1209.428589,1064.233398,1207.112183
...,...,...,...,...,...
1801,2022-10-15,1297.305908,1300.745972,1268.468506,1274.871704
1802,2022-10-16,1275.005493,1312.634521,1275.005493,1306.296631
1803,2022-10-17,1306.309570,1335.647827,1297.447266,1331.713623
1804,2022-10-18,1331.669922,1339.085938,1291.660645,1310.447021


In [7]:
del df_eth['Date']
scaler = MinMaxScaler(feature_range=(0,1))
df_eth = scaler.fit_transform(df_eth)

with open('scaler ETH.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# split dataset into train and test split
training_size = int(len(df_eth)*0.8)
test_size = len(df_eth) - training_size

train_data, test_data = df_eth[0:training_size,:], df_eth[training_size:len(df_eth),]

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), :]
        dataX.append(a)
        dataY.append(dataset[i + time_step,-1])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 15
X_train_wf, y_train_wf = create_dataset(train_data, time_step)
X_test_wf, y_test_wf = create_dataset(test_data, time_step)

print("X_train: ", X_train_wf.shape)
print("y_train: ", y_train_wf.shape)
print("X_test: ", X_test_wf.shape)
print("y_test: ", y_test_wf.shape)

In [ ]:
model_wf=Sequential()
model_wf.add(LSTM(10,input_shape=(None,4),activation="relu"))
model_wf.add(Dense(1))
model_wf.compile(loss="mean_squared_error",optimizer="adam")
history = model_wf.fit(
                    X_train_wf,
                    y_train_wf,
                    validation_data=(X_test_wf,y_test_wf),
                    epochs=200,
                    batch_size=32,
                    verbose=1
                    )
model_wf.evaluate(X_test_wf, y_test_wf)

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model_wf.save('model_wf ETH.h5')
model_wof.save('model_wof ETH.h5')